# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

'pytor'

# Result Filename

In [2]:
result_filename='ms_CelebDPR.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob
from pprint import pprint

## API Setting 

In [4]:
subscription_key = "3d726d64c2a94ee4bfde80b28a202151"
endpoint="https://cv-adf.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "5a2a12a86c6c4bb6b8c4df003cec53d6"
endpoint="https://face-adf.cognitiveservices.azure.com/"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

## CelebDPR

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [7]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(single_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    multi_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(multi_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', multi_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [14]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/*")
count=0
skip=0
file_count=0
actors_list.sort()
for actor in actors_list:
    print()
    count+=1
#     if skip ==0:
#         if os.path.basename(actor) =="June Lapine Aka Shoe0nHead":
#             skip=1
#         else:
#             continue
#     print(os.path.basename(actor))
#     break
    
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real",
                                os.path.basename(actor)+".jpg")
    fake_images_list=glob(os.path.join(actor,"*"))
    print(real_image)
#     pprint(fake_images_list)
#     continue
    result_real_pred,result_real_conf=domain_specific_content(real_image)
    real_des=describe_image(real_image)
    src=get_name(real_image)
    file_count+=1
    for fake_image in fake_images_list:
        file_count+=1
        print(fake_image)
#         continue
        result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
        fake_des=describe_image(fake_image)
        tar=get_name(fake_image)
        similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity,real_des,fake_des])
        file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total Celebrities:",count)
print("Total Files:",file_count)


/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real/Alexandra Daddario.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/87_00468_0.jpg
No celebrities detected.
No similar faces found in 87_00468_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/49_00606_0.jpg
No celebrities detected.
No similar faces found in 49_00606_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/55_00833_0.jpg
No celebrities detected.
No similar faces found in 55_00833_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/18_01099_0 - Copy.jpg
No celebrities detected.
No similar faces found in 18_01099_0 - Copy.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/30_00077_0.jpg
No celebrities detected.
No similar faces found in 30_00077_0.jpg .
/home/sha/project/adversarial_deepfake/src/S

No celebrities detected.
No similar faces found in 25_00420_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/67_00541_0.jpg
No celebrities detected.
No similar faces found in 67_00541_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/79_00751_0.jpg
No celebrities detected.
No similar faces found in 79_00751_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/6_01031_0.jpg
No celebrities detected.
No similar faces found in 6_01031_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/82_00519_0.jpg
No celebrities detected.
No similar faces found in 82_00519_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/90_00406_0.jpg
No celebrities detected.
No similar faces found in 90_00406_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddar

No celebrities detected.
No similar faces found in 57_00886_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/45_01084_0 - Copy.jpg
No celebrities detected.
No similar faces found in 45_01084_0 - Copy.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/47_00174_0.jpg
No celebrities detected.
No similar faces found in 47_00174_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/42_00646_0.jpg
No celebrities detected.
No similar faces found in 42_00646_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexandra Daddario/78_00682_0.jpg
No celebrities detected.
No similar faces found in 78_00682_0.jpg .

/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real/Alexis Bledel.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/77_02884_0.jpg
No celebrities detected.
/home/sha/project/adversarial

No celebrities detected.
No similar faces found in 21_02361_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/37_00793_0.jpg
No celebrities detected.
No similar faces found in 37_00793_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/13_01591_0.jpg
No celebrities detected.
No similar faces found in 13_01591_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/76_00091_0.jpg
No celebrities detected.
No similar faces found in 76_00091_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/23_01774_0.jpg
No celebrities detected.
No similar faces found in 23_01774_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/67_00069_0.jpg
No celebrities detected.
No similar faces found in 67_00069_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/20_01090_0.jpg
No similar

No celebrities detected.
No similar faces found in 97_00160_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/38_00652_0.jpg
No celebrities detected.
No similar faces found in 38_00652_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Alexis Bledel/98_02188_0.jpg
No similar faces found in 98_02188_0.jpg .

/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real/Angelina Jolie.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Angelina Jolie/25_01977_0.jpg
No celebrities detected.
No similar faces found in 25_01977_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Angelina Jolie/32_02733_0.jpg
No celebrities detected.
No similar faces found in 32_02733_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Angelina Jolie/94_03849_0.jpg
No celebrities detected.
No similar faces found in 94_03849_0.jpg .
/home/sha/project/adversarial_deepfake/sr

...

No similar faces found in 1_01584_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/67_01835_0.jpg
No celebrities detected.
No similar faces found in 67_01835_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/75_02295_0.jpg
No celebrities detected.
No similar faces found in 75_02295_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/93_01170_0.jpg
No celebrities detected.
No similar faces found in 93_01170_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/89_01975_0.jpg
No celebrities detected.
No similar faces found in 89_01975_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/33_01962_0.jpg
No celebrities detected.
No similar faces found in 33_01962_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Sophie Turner/99_02295_0 - Copy.jpg
No celebrities detected.
No s

No similar faces found in 12_01559_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/8_02873_0.jpg
No celebrities detected.
No similar faces found in 8_02873_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/19_01375_0.jpg
No celebrities detected.
No similar faces found in 19_01375_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/7_01326_0.jpg
No celebrities detected.
No similar faces found in 7_01326_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/30_03108_0.jpg
No celebrities detected.
No similar faces found in 30_03108_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/67_02848_0.jpg
No celebrities detected.
No similar faces found in 67_02848_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/33_01282_0.jpg
No celebrities detected.
No similar faces fou

No celebrities detected.
No similar faces found in 87_01321_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/36_01673_0.jpg
No celebrities detected.
No similar faces found in 36_01673_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/93_00538_0.jpg
No celebrities detected.
No similar faces found in 93_00538_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/70_01681_0.jpg
No celebrities detected.
No similar faces found in 70_01681_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/83_03209_0.jpg
No celebrities detected.
No similar faces found in 83_03209_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/45_02832_0.jpg
No celebrities detected.
No similar faces found in 45_02832_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/Taylor Swift/13_02060_0.jpg
No celebrities d

No celebrities detected.
No similar faces found in 29_01336_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/67_01917_0.jpg
No celebrities detected.
No similar faces found in 67_01917_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/47_01812_0.jpg
No celebrities detected.
No face detected from image 47_01812_0.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/79_02040_0.jpg
No celebrities detected.
No similar faces found in 79_02040_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/50_00815_0.jpg
No celebrities detected.
No similar faces found in 50_00815_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/46_01653_0.jpg
No celebrities detected.
No similar faces found in 46_01653_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/71_00875_0.jpg
No celebrities detected.
No similar face

No celebrities detected.
No similar faces found in 35_01600_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/82_01091_0.jpg
No celebrities detected.
No similar faces found in 82_01091_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/64_01574_0.jpg
No celebrities detected.
No similar faces found in 64_01574_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/88_01866_0.jpg
No celebrities detected.
No similar faces found in 88_01866_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/56_01794_0.jpg
No celebrities detected.
No similar faces found in 56_01794_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/10_01508_0.jpg
No celebrities detected.
No similar faces found in 10_01508_0.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/iJustine/52_02093_0.jpg
No celebrities detected.
No face detecte